In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [6]:
import os, sys, pprint

path_mod = os.getcwd().replace("\\src\\spark-sql", "")
sys.path.insert(0,path_mod)

from src.connections import connect

context = connect.ps_connection("stack_overflow", "ERROR", 3)
session = context.sql

In [11]:
dataFrameReader = session.read

responses = (
    dataFrameReader.option("header", "true")
    .option("inferSchema", value = True)
    .csv("2016-stack-overflow-survey-responses.csv")
)

responses.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- collector: string (nullable = true)
 |-- country: string (nullable = true)
 |-- un_subregion: string (nullable = true)
 |-- so_region: string (nullable = true)
 |-- age_range: string (nullable = true)
 |-- age_midpoint: double (nullable = true)
 |-- gender: string (nullable = true)
 |-- self_identification: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- occupation_group: string (nullable = true)
 |-- experience_range: string (nullable = true)
 |-- experience_midpoint: double (nullable = true)
 |-- salary_range: string (nullable = true)
 |-- salary_midpoint: double (nullable = true)
 |-- big_mac_index: double (nullable = true)
 |-- tech_do: string (nullable = true)
 |-- tech_want: string (nullable = true)
 |-- aliens: string (nullable = true)
 |-- programming_ability: double (nullable = true)
 |-- employment_status: string (nullable = true)
 |-- industry: string (nullable = true)
 |-- company_size_range: string (null

In [13]:
AGE_MIDPOINT = "age_midpoint"
SALARY_MIDPOINT = "salary_midpoint"
SALARY_MIDPOINT_BUCKET = "salary_midpoint_bucket"

responseWithSelectedColumns = responses.select("country", "occupation", AGE_MIDPOINT, SALARY_MIDPOINT)
responseWithSelectedColumns.show()

+-----------+--------------------+------------+---------------+
|    country|          occupation|age_midpoint|salary_midpoint|
+-----------+--------------------+------------+---------------+
|Afghanistan|                null|        22.0|           null|
|Afghanistan|Mobile developer ...|        32.0|        45000.0|
|Afghanistan|                null|        null|           null|
|Afghanistan|              DevOps|        null|         5000.0|
|Afghanistan|                null|        65.0|           null|
|Afghanistan|                null|        22.0|           null|
|Afghanistan|       Growth hacker|        null|       210000.0|
|Afghanistan|Back-end web deve...|        27.0|         5000.0|
|    Albania|                null|        27.0|           null|
|    Albania|Back-end web deve...|        22.0|         5000.0|
|    Albania|Full-stack web de...|        27.0|         5000.0|
|    Albania|Full-stack web de...|        22.0|        15000.0|
|    Albania|Full-stack web de...|      

In [14]:
responseWithSelectedColumns.filter(responseWithSelectedColumns['country'] == "Afghanistan").show()

+-----------+--------------------+------------+---------------+
|    country|          occupation|age_midpoint|salary_midpoint|
+-----------+--------------------+------------+---------------+
|Afghanistan|                null|        22.0|           null|
|Afghanistan|Mobile developer ...|        32.0|        45000.0|
|Afghanistan|                null|        null|           null|
|Afghanistan|              DevOps|        null|         5000.0|
|Afghanistan|                null|        65.0|           null|
|Afghanistan|                null|        22.0|           null|
|Afghanistan|       Growth hacker|        null|       210000.0|
|Afghanistan|Back-end web deve...|        27.0|         5000.0|
+-----------+--------------------+------------+---------------+



In [15]:
responseWithSelectedColumns.filter(responseWithSelectedColumns.country == "Afghanistan").show()

+-----------+--------------------+------------+---------------+
|    country|          occupation|age_midpoint|salary_midpoint|
+-----------+--------------------+------------+---------------+
|Afghanistan|                null|        22.0|           null|
|Afghanistan|Mobile developer ...|        32.0|        45000.0|
|Afghanistan|                null|        null|           null|
|Afghanistan|              DevOps|        null|         5000.0|
|Afghanistan|                null|        65.0|           null|
|Afghanistan|                null|        22.0|           null|
|Afghanistan|       Growth hacker|        null|       210000.0|
|Afghanistan|Back-end web deve...|        27.0|         5000.0|
+-----------+--------------------+------------+---------------+



In [16]:
groupedData = responseWithSelectedColumns.groupBy("occupation")
groupedData.count().show()

+--------------------+-----+
|          occupation|count|
+--------------------+-----+
|     Product manager|   18|
|Business intellig...|    8|
|Mobile developer ...|    3|
|System administrator|   34|
|             Student|  234|
|    Mobile developer|   60|
| Engineering manager|   22|
|                null|  297|
|            Designer|   15|
|Embedded applicat...|   33|
| Graphics programmer|   11|
|               other|   67|
|   Desktop developer|   87|
|Developer with a ...|   25|
|       Growth hacker|    5|
|   Quality Assurance|   10|
|             Analyst|   20|
|Full-stack web de...|  498|
|Mobile developer ...|   32|
|Machine learning ...|   10|
+--------------------+-----+
only showing top 20 rows



In [17]:
responseWithSelectedColumns.filter(responseWithSelectedColumns[AGE_MIDPOINT] < 20).show()

+---------+--------------------+------------+---------------+
|  country|          occupation|age_midpoint|salary_midpoint|
+---------+--------------------+------------+---------------+
|  Algeria|             Student|        16.0|           null|
|  Algeria|Back-end web deve...|        16.0|           null|
|Argentina|             Student|        16.0|         5000.0|
|Argentina|Back-end web deve...|        16.0|         5000.0|
|  Armenia|Back-end web deve...|        16.0|         5000.0|
|  Armenia|                null|        16.0|           null|
|  Armenia|Mobile developer ...|        16.0|         5000.0|
|  Armenia|Mobile developer ...|        16.0|         5000.0|
|  Austria|Mobile developer ...|        16.0|           null|
|  Austria|Full-stack web de...|        16.0|           null|
|  Austria|Full-stack web de...|        16.0|        15000.0|
|  Austria|                null|        16.0|           null|
|  Austria|             Student|        16.0|           null|
|  Austr

In [18]:
responseWithSelectedColumns.orderBy(responseWithSelectedColumns[SALARY_MIDPOINT], ascending = False).show()

+------------------+--------------------+------------+---------------+
|           country|          occupation|age_midpoint|salary_midpoint|
+------------------+--------------------+------------+---------------+
|         Argentina|Back-end web deve...|        32.0|       210000.0|
|           Denmark|              DevOps|        44.5|       210000.0|
|         Argentina|Full-stack web de...|        27.0|       210000.0|
|           Denmark|Enterprise level ...|        32.0|       210000.0|
|Dominican Republic|Executive (VP of ...|        37.0|       210000.0|
|             China|Machine learning ...|        22.0|       210000.0|
|            France|Full-stack web de...|        32.0|       210000.0|
|           Denmark|Full-stack web de...|        22.0|       210000.0|
|       Afghanistan|       Growth hacker|        null|       210000.0|
|          Bulgaria|Enterprise level ...|        37.0|       195000.0|
|           Denmark|Full-stack web de...|        32.0|       185000.0|
|     

In [19]:
responseWithSelectedColumns.groupBy("country").avg(SALARY_MIDPOINT).show()

+------------------+--------------------+
|           country|avg(salary_midpoint)|
+------------------+--------------------+
|           Germany|  46491.228070175435|
|       Afghanistan|             66250.0|
|          Cambodia|              5000.0|
|            France|  39648.760330578516|
|           Algeria|             30000.0|
|         Argentina|  27950.819672131147|
|           Belgium|   45989.01098901099|
|           Ecuador|             40000.0|
|           Albania|   8333.333333333334|
|           Finland|   45714.28571428572|
|           Bahamas|             95000.0|
|             China|             54687.5|
|           Belarus|             10000.0|
|             Chile|  41666.666666666664|
|           Croatia|  14166.666666666666|
|           Andorra|             40000.0|
|           Bolivia|              5000.0|
|           Denmark|   68768.65671641791|
|        Bangladesh|   7307.692307692308|
|Bosnia Herzegovina|             10000.0|
+------------------+--------------

In [35]:
responsesWithSalaryBucket = (
    responses.filter(responses[SALARY_MIDPOINT] > 0)
    .withColumn(SALARY_MIDPOINT_BUCKET, (responses[SALARY_MIDPOINT] / 20000).cast("int") * 20000)
)

groupedCount = responsesWithSalaryBucket.groupby(SALARY_MIDPOINT_BUCKET).count()
(
    groupedCount.withColumn("percent", groupedCount['count'] / responsesWithSalaryBucket.count() * 100)
    .orderBy(SALARY_MIDPOINT_BUCKET, ascending = False)
    .show()
)

+----------------------+-----+-------------------+
|salary_midpoint_bucket|count|            percent|
+----------------------+-----+-------------------+
|                200000|    9| 0.6280530355896721|
|                180000|    3|0.20935101186322402|
|                160000|    5|0.34891835310537334|
|                140000|   11| 0.7676203768318213|
|                120000|   23| 1.6050244242847174|
|                100000|   51|  3.558967201674808|
|                 80000|   63| 4.3963712491277045|
|                 60000|  134|  9.351011863224006|
|                 40000|  260| 18.143754361479413|
|                 20000|  351|  24.49406838799721|
|                     0|  523|  36.49685973482205|
+----------------------+-----+-------------------+



In [36]:
session.stop()